<h1><center>Dynamic Programming Algorithm to solve the Shortest Path Problem with Time Constraints on Movement and Parking (Sancho 1992)</center></h1><h3><center>Implemented by Laurin Luttmann and Jan-Niklas Anders</center></h3>

Import of necessary packages

In [1]:
from collections import defaultdict
import time
import pandas as pd

### Definition of the example

Define the graph G. The graph has nodes and arcs. To each node a time interval is assigned which specifies the time during which parking is allowed. Each arcs has a distance, which is in this case the duration to travel between the corresponding nodes. Moreover, the predecessors and successors of each node are saved in a separate class attribute.

In [2]:
class Graph:
    """
    The Graph class contains the network of the example specified in the function build_graph. Nodes, arcs, 
    direct predecessors as well as successors of nodes, distances between nodes and the time windows for 
    parking and departure are attributes of this class object.
    """
    def __init__(self):
        self.nodes = set()
        self.edges = list()
        self.edges_out = defaultdict(list)
        self.edges_in = defaultdict(list)
        self.distance = dict()
        self.f_dep_t = dict()
        self.f_park_t = dict()
        self.build_graph()

    def add_node(self, value, feasible_parking_time):
        self.nodes.add(value)
        self.f_park_t[value] = feasible_parking_time

    def add_edge(self, from_node, to_node, distance, feasible_dep_time):
        self.edges.append((from_node, to_node))
        self.edges_out[from_node].append(to_node)
        self.edges_in[to_node].append(from_node)
        self.distance[(from_node, to_node)] = distance
        self.f_dep_t[(from_node, to_node)] = feasible_dep_time
        
    def build_graph(self):
        """
        Define the network for the example in this function. Note that an additional "artifical"
        node has to be specified at the very end of the network, where no restrictions in terms of
        parking exist. This is needed to calculate the earliest arrival time at the sink.
        :return: graph containing the data of the specified example
        """
        # define the feasible parking times at each node
        parking_times = {
            1: [0, float('inf')],
            2: [18, 30],
            3: [35, 45],
            4: [64, 75],
            5: [35, 50],
            6: [64, 75],
            7: [0, float('inf')],
            8: [0,0] # artificial node
        }

        # define the feasible travel times between all nodes
        travel_times = {
            (1, 2): [10, 15],
            (1, 3): [10, 25],
            (1, 5): [15, 25],
            (2, 4): [28, 35],
            (2, 6): [26, 34], 
            (2, 7): [32, 60],
            (3, 2): [35, 50],
            (3, 4): [43, 50],
            (4, 7): [70, 99],
            (5, 2): [30, 80],
            (5, 6): [48, 55],
            (6, 7): [72, 80],
            (7, 8): [0, 0] # artificial node
        }

        # define the duration to travel between nodes
        duration = {
            (1, 2): 10,
            (1, 3): 20,
            (1, 5): 25,
            (2, 4): 35,
            (2, 6): 35, 
            (2, 7): 30,
            (3, 2): 15,
            (3, 4): 20,
            (4, 7): 30,
            (5, 2): 20,
            (5, 6): 25,
            (6, 7): 10,
            (7, 8): 0 # artificial node
        }

        arcs = list(duration.keys())

        # initialize the nodes
        for i in range(1, 9):
            self.add_node(i, parking_times[i])

        # Arcs of the example
        for arc in arcs:
            self.add_edge(*arc, duration[arc], travel_times[arc]) # (from node, to node, distance/duration, feasible travel times)

### Define some helper functions

The function **get_route_for_arc** returns all arcs used in order to arrive at a given arc for a specified solution set

In [3]:
def get_route_for_arc(arc: tuple, optimal_i: dict):
    """
    This function returns all arcs used in order to arrive at a given arc for a specified solution set
    :param arc: The arc for which preceeding arcs shall be determined
    :param optimal_i: The current solution found by the algorithm, i.e. for all f(i,j) the optimal
    preceeding node.
    :return: Set of arcs preceeding the arc specified in the functions arguments
    """
    check_route_for = arc
    route = list()
    i = check_route_for[0]
    j = check_route_for[1]
    try:
        while True:
            curr_arc = (i, j)
            i = optimal_i[curr_arc]
            j = curr_arc[0]
            route.append((i, j))
            if i == 1:
                break
        return [i for i in reversed(route)]
    # Direct links from source nodes have no optimal predecessor, hence they don´t appear in the solution
    # set. Therefore, when keys are not found in the dictionary, an empty list will be returned
    except KeyError:
        return []

The function **get_time_windows**  returns the time windows for a given node i and a node j to travel next to.

In [4]:
def get_time_windows(g, i, j):
    """
    This function returns the time windows for a given node i and a node j to travel next to.
    :param g: The Graph object
    :param i: Some node i for which the time window for parking must be retrieved
    :param j: Some node j to which the possible departure times from i shall be retrieved
    preceeding node.
    :return: The time windows for parking and movement
    """
    # specify time window for parking
    alpha_i = g.f_park_t[i][0]
    beta_i = g.f_park_t[i][1]

    # specify time window for traveling the arc
    gamma_ij = g.f_dep_t[(i, j)][0]
    delta_ij = g.f_dep_t[(i, j)][1]

    return alpha_i, beta_i, gamma_ij, delta_ij

## Implementation of the algorithm

The function **calc_min_earliest_arrival_time** determines for a given graph $G$ the earliest feasible arrival time $f(j, k)$ at a node $j$. This is done by first iterating over all nodes that have a direct connection to node $j$. In each iteration, the dynamic programming technique is used, by retrieving the earliest feasible arrival time at a given node $i$ which has been determined earlier. This algorithm hence has to run sequencially, as for a given node $j$ the earliest feasible arrival times of all its predecessors have to be calculated alread. Note here, that $f(1,j)=0$ (Sancho, p. 195)

The $f(i, j)$ is then used to determine the parking time at $i$. This is done using the function **check_for_feasibility** which will be explained in the furth course of this script.

With the parking time $p_i$ at $i$, the earliest feasible arrival time at this node as well as the distance between $i$ and $j$ the arrival time at $j$ where $i$ is the predecessor of $j$ can be determined: $f'(j,k) = f(i,j) + p_i + t_{ij}$, where $f'(j,k)$ refers to a potential solution to the problem $\underset{i}{min}[f(i,j) + p_i + t_{ij}]$

In the end, the minimum of the potential solutions is selected and the results for $f(j,k)$, $p_i$ and $opt\,i$ are saved. This can be seen as the **pareto optimal solution** for the subprblem of traveling from $i$ to $j$.

In [5]:
def calc_min_earliest_arrival_time(g, j, k, earliest_feasible_arrival_times, opt_is, parking, blacklist):
    """
    :param g: graph
    :param i: node i
    :param j: node j
    :param earliest_feasible_arrival_times: dictionary containing the f(i,j) found so far
    :param opt_is: dictionary containing the opt i (optimal predecessors) found so far
    :param parking: dictionary containing the parking times determined so far
    :param blacklist: dictionary containing the routes that may not be used as they yield infeasible solutions
    :return: 
    """
    if j == 1:
        # we have initialized all arcs from the source already
        return earliest_feasible_arrival_times, opt_is, parking, blacklist
    
    pot_earliest_feasible_arrival_times = []
    inter_parking = []
    print(g.edges)
    arcs_to_j = [ij[0] for ij in g.edges if ij[1] == j]
    print(arcs_to_j)
    print("The following nodes move into {}: {}".format(j, arcs_to_j))
    for i in arcs_to_j:
        if (i, j) in blacklist[j, k]:
            print("Arc ({},{}) is blacklisted for route containing current arc ({},{})".format(i, j, j, k))
            pot_earliest_feasible_arrival_times.append(float('inf'))
            inter_parking.append(float('inf'))
        else:
            print("f({}, {}) is: {}".format(i, j, earliest_feasible_arrival_times[i, j][-1]))
            p_i = check_for_feasibility(g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist)
            t_ij = g.distance[i, j]
            inter_parking.append(p_i)
            if p_i is not False and type(p_i) == int:
                print("f({},{}): {}; p_{}: {}; t_{}{}: {}".format(i, j, earliest_feasible_arrival_times[i, j][-1],
                                                                  i, p_i, i, j, t_ij))
                pot_earliest_feasible_arrival_times.append(earliest_feasible_arrival_times[i, j][-1] + p_i + t_ij)
            # if the backtracking founds that an arc doen't yield a feasible solution, it returns False
            else:
                pot_earliest_feasible_arrival_times.append(float('inf'))
        print("pot_earliest_feasible_arrival_times: ", pot_earliest_feasible_arrival_times)
    min_arrival_time_index = pot_earliest_feasible_arrival_times.index(min(pot_earliest_feasible_arrival_times))
    earliest_feasible_arrival_times[j, k].append(pot_earliest_feasible_arrival_times[min_arrival_time_index])
    parking[j, k].append(inter_parking[min_arrival_time_index])
    opt_is[j, k] = arcs_to_j[min_arrival_time_index]
    print("optimal i for ({},{}): {}".format(j, k, opt_is[j, k]))
    
    return earliest_feasible_arrival_times, opt_is, parking, blacklist

### Domination criteria implementation

In the function **check_for_feasibility** the rules described by Sancho on page 194 are implemented. This implementation yields a domination criteria, which reduces the solution space in the following way:

1.) if parking is feasible, i.e. the arrival time at a node falls withing the time window for parking at this node, the **smallest feasible parking time is chosen**: Using this greedy approach might lead to infeasible solutions on nodes visited later on, however through backtracking the parking times found earlier may be increased. This is also done in a greedy fashion, as will be described in the function that implements the backtracking.

2.) infeasible solutions which cannot be made feasible through increasing the arrival time are omitted (i. e. if the condition of eq. (5) is met).

3.) situations where parking is not possible but the route ahead is already open, are detected and directly assigned a parking time of zero.

In [6]:
def check_for_feasibility(g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist):
    """
    Determines the parking time
    :param g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist: see above
    :return: 
    
    """
    print(i, j, earliest_feasible_arrival_times)
    earliest_feasible_at = earliest_feasible_arrival_times[i, j][-1]
    print("Check feasibility for usage of arc ({}) for earliest feasible time {}".format((i, j), earliest_feasible_at))
    if earliest_feasible_at == float('inf'):
        return float('inf')

    # get time window for parking and for traveling the arc
    alpha_i, beta_i, gamma_ij, delta_ij = get_time_windows(g, i, j)

    # check first equation:
    # in the first equation of the paper, parking is considered to be possible. Hence, inside the following
    # for loop, the feasible parking times are determined. If the resulting set is empty, the algorithm proceeds
    # to the next condition. To determine if a parking time is feasible, the conditions in the first equation
    # of the paper are checked.
    max_p_i = min(beta_i - earliest_feasible_at, delta_ij - earliest_feasible_at)
    feasible_p_i = []
    for pot_p_i in range(max_p_i + 1):
        # here, eq. (1) is checked.
        if (alpha_i <= earliest_feasible_at <= beta_i) and (gamma_ij <= earliest_feasible_at + pot_p_i <= delta_ij):
            feasible_p_i.append(pot_p_i)
    print("feasible parking times at {} when moving to {}: {}".format(i, j, feasible_p_i))
    if len(feasible_p_i) > 0:
        print("1st cond satisfied")
        # in here, use greedy methodology and choose the smallest feasible parking time. If this leeds
        # to an infeasible solution in later stages, the backtracking will try to increase the parking time.
        p_i = min(feasible_p_i)

    # check second equation
    elif (beta_i - earliest_feasible_at <= 0) and (gamma_ij <= earliest_feasible_at <= delta_ij):
        print("2nd cond satisfied")
        p_i = 0

    # check third equation
    # note: in comparison to the paper by Sancho, this condition has been changed, as the original
    # set of conditions did not cover all cases. Here, it is checked WHETHER
    # 1.) we arrive to early at a node, so that the route ahead is blocked and parking is not possible yet OR
    # 2.) parking is not possible, as beta is smaller than gamma, and we arrive to early to directly take the route ahead.
    # If this condition is satisfied, the backtracking is done to check whether longer parking at predecessor nodes
    # makes the current route feasible.
    elif (earliest_feasible_at < alpha_i and earliest_feasible_at < gamma_ij) or (
            earliest_feasible_at < gamma_ij and beta_i < gamma_ij):
        print("3rd condition satisfied")
        p_i = do_recursion(g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist)
        print("recursion is done, result is: ", p_i)

    # check fourth equation
    elif (gamma_ij < earliest_feasible_at < alpha_i <= delta_ij) or (
            gamma_ij < earliest_feasible_at < delta_ij <= alpha_i):
        print("4th cond satisfied")
        p_i = 0

    # check fifth equation
    elif delta_ij < earliest_feasible_at:
        print("5th cond satisfied")
        p_i = float('inf')

    # raise an error if a case satisfies no condition
    else:
        raise AssertionError("Some cases are not considered in feasibility check")

    return p_i

### Implementation of the Backtracking

This function implements the backtracking which is mentioned in the paper by Sancho on page 195. This function will be called from the function **check_if_feasible** if an infeasible solution is detected, which can be made feasible by increasing parking at predecessor nodes. If no additional time is mentioned, the function determines the additional parking time needed at the direct predecessor by itself. This is done in a **greedy** manner, by taking the minimum time so that either $f(i,j) = \gamma_{ij}$ or $f(i,j) = \alpha_i$.  

If the additional parking time imposed at the predecessor node yields a feasible solution, the earliest feasible arrival time $f(i,j)$ is recalculated with the new parameters and it's feasibility will also be checked. If no further infeasible solution is deteced, the recursion will just return the new parking time at the node $i$ for $f(i,j)$. 

If the additional parking time again leads to an infeasible solution, e. g. because the time window for parking at the predecessor node is exceeded, the function will be called again from **add_parking_to_predecessor**, but now with the predecessor node of the current predecessor (2nd order predecessor) and the parking time needed at the 2nd order predecessor in order to make the solution feasible. This procedure is repeated 

1.) either until a feasible solution is found, in which case the solution found so far is updated or

2.) until the algorithm arrived at the source node, which has no predecessor. In this case, the route that was backtracked is infeasible and it´s arcs will be set on a "blacklist", so that this particular route will not be considered by the algorithm anymore. Then, the optimal route to node $(i,j)$ is determined.

In [7]:
def do_recursion(g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist, time_gap=None):
    """
    Recursion for identifying a parking time p_i when in succeeding links f(j,k) a situation occurs,
    where condition (equation) 3 from the paper is met, i.e. a node is visited before the parking
    is allowed as well as before the transition to node k is possible. In this case, parking time
    in preceeding nodes has to be increased. This works in a backtracking manner, i.e. if parking
    cannot increased in preceeding node i because parking is not allowed there yet but transition
    to the next node j is, then the preceeding node for i is checked. If at no preceeding node
    parking is possible, then the solution set will be omitted (set to infinity)
    :param g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist: see above
    :param blacklist: dictionary containing the routes that may not be used as they yield infeasible solutions
    :time_gap: the additional time that is needed so that at the node either parking will be possible or 
    the route ahead will open
    :return: Recursion: Calls again check_if_feasible with the new parking time for the predecessor node. If that is still
    infeasible, the recursion is repeated until a feasible solution is found or the node that is currently under 
    observation has no predecessors. In this case, the recursion will return infinity, leading to a solution which 
    will never be selected.
    """
    earliest_feasible_at = earliest_feasible_arrival_times[i, j][-1]
    if earliest_feasible_at == float('inf'):
        return float('inf')
    # specify time window for parking
    print("so lets do the backtracking")
    alpha_i = g.f_park_t[i][0]

    # specify time window for traveling the arc
    gamma_ij = g.f_dep_t[(i, j)][0]

    previous_arcs = get_route_for_arc((i, j), opt_is)
    print("The arcs preceeding arc ({},{}) are: {}".format(i, j, previous_arcs))
    all_blacklisted = all([x in blacklist[i, j] for x in previous_arcs])
    if len(previous_arcs) == 0 or all_blacklisted:
        return float('inf')
    else:
        # this time is missing on node j in order to be able to park or directly move to k.
        # The time that is missing must be greater (or equal to zero), hence this condition is applied here.
        # Generally, the least time needed to make a solution feasible is seeked
        if not time_gap:
            time_missing = [alpha_i - earliest_feasible_at, gamma_ij - earliest_feasible_at]
            time_gap = min([x for x in time_missing if x >= 0])
            if time_gap == alpha_i - earliest_feasible_at:
                print("{} time periods are missing to be able to park at the node".format(time_gap))
            else:
                print("{}time periods are missing to be able to move along the route ahead".format(time_gap))
        curr_prev_arc = previous_arcs[-1]
        # for curr_prev_arc in reversed(previous_arcs):
        print(curr_prev_arc)
        f_i1_i2 = earliest_feasible_arrival_times[curr_prev_arc][-1]
        new_p_i1 = add_parking_to_predecessor(g, *curr_prev_arc, earliest_feasible_arrival_times, opt_is, 
                                             parking, blacklist, add_park_time=time_gap, successors=(i, j))
        # check_for_feasibility returns infinity if parking cannot be extended by the amount of time
        # specified in time_gap at node i1. If that is not the case, the solution is feasible and the
        # algorithm may proceed with the new parking time at the node i1.
        if not new_p_i1 == float('inf'):
            new_earliest_feasible_at = f_i1_i2 + new_p_i1 + g.distance[curr_prev_arc]
            print("new f({}): ".format((i, j)), new_earliest_feasible_at, "new p_{}: ".format(i), new_p_i1)
            earliest_feasible_arrival_times[i, j].append(new_earliest_feasible_at)
            parking[i, j].append(new_p_i1)
            return check_for_feasibility(g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist)
        # else, the backtracking needs to proceed further to see if the time at pr
        else:
            # put the arc for this specific route on the blacklist so it wont be used anymore
            blacklist[i, j].append(curr_prev_arc)
            print("Infeasible solution found. Recalculate the best path for node {} proceeding to {}".format(i, j))
            calc_min_earliest_arrival_time(g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist)
            return check_for_feasibility(g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist)

The function **add_parking_to_predecessor** adds the additional parking time needed at the successor to make the route a feasible solution to the parking time of the predecessor found in the previous solution. If that is a feasible solution, the new parking time will be returned and the current solution is updated.

If it is not feasible, the backtracking (**do_recursion**) will be called again, now with the predecessor nodes and the new parking time. Note that parking at the 2nd order predecessor will not make parking feasible, however, the increase of the parking time at the 2nd, 3rd.... predecessors might lead to a solution where the routes ahead are all open.

In [8]:
def add_parking_to_predecessor(g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist, add_park_time, successors):
    earliest_feasible_at = earliest_feasible_arrival_times[i, j][-1]

    alpha_i, beta_i, gamma_ij, delta_ij = get_time_windows(g, i, j)

    max_p_i = min(beta_i - earliest_feasible_at, delta_ij - earliest_feasible_at)
    if add_park_time:
        print("additional parking time at node {} is needed: ".format(i), add_park_time)
        # if we run this function in backtracking mode, where at preceeding nodes the parking time has to be
        # extended, the needed additional time is added to the parking time found in the previous solution
        # parking time at the corresponding node.
        new_p_i = parking[successors][-1] + add_park_time
        print("This sums up to: ", new_p_i)
        if new_p_i <= max_p_i:
            print('which is still a feasible solution. Proceed with p_{} = {}'.format(i, new_p_i))
            return new_p_i
        else:
            print("which is not a feasible solution. ")
            # if the parking time with the additional time exceeds the time window where parking is allowed, we
            # arrived again at an infeasible solution, hence the backtracking will proceed to the next predecessor,
            # until a solution is found or no more predecessors exist (which is the case for the source node).
            # The parking time at the next predecessor must match the parking time at the node found in the
            # previous solution plus the additional parking time.
            print("Another round of recusion will be tried. The parking at the next node needs to be increased \n "
                  "by add_park_time: {} + current paarking: {}".format(add_park_time, parking[successors][-1]))
            return do_recursion(g, i, j, earliest_feasible_arrival_times, opt_is, parking, blacklist, time_gap=new_p_i)

### Main function

This function runs the dynamic programming algorithm sequentially. Hence, at first the sink node is initialized with $f(i,j)$ being set to zero for all arcs moving out of the sink. After that, the algorithm iterates over all succeeding nodes in ascending order and performs the dynamic programming technique as described for function **calc_min_earliest_arrival_time**

In [9]:
def main(g: Graph):
    """
    Runs the algorithm. First initializes all f(1,k) with 0. After that, iterates over all other nodes of the 
    graph in ascending order.
    :param g: the graph g as object of class Graph
    :return: the solution, consisting of the earliest feasible arrival times, optimal parking durations at each node
    and the optimal predecessor for each node. Additionally (in contrast to the representation of the solution by Sancho),
    the arcs leading to an infeasible solution in a specific route are returned
    """
    earliest_feasible_arrival_times = defaultdict(list)
    source = min(g.nodes)
    terminal = max(g.nodes)
    optimal_predecessors = dict()
    optimal_parking = defaultdict(list)
    blacklist = defaultdict(list)
    arcs_from_source = [jk[1] for jk in g.edges if jk[0] == source]
    print("initialize the arcs from the source")    
    for k in arcs_from_source:
        earliest_feasible_arrival_times[source, k].append(0)
    for k in g.nodes.difference({source}):
        print("The iteration over k is currently at: ", k)
        arcs_to_k = [jk[0] for jk in g.edges if jk[1] == k]
        print("The following nodes move into {}: {}".format(k, arcs_to_k))
        for j in arcs_to_k:
            earliest_feasible_arrival_times, optimal_predecessors, optimal_parking, blacklist =  \
            calc_min_earliest_arrival_time(g, j, k, earliest_feasible_arrival_times, optimal_predecessors, 
                                           optimal_parking, blacklist)

    return earliest_feasible_arrival_times, optimal_predecessors, optimal_parking, blacklist

In [10]:
if __name__ == "__main__":
    # stop runtime for algorithm
    start = time.time()
    # initialize Graph G
    g = Graph()
    # run algorithm
    earliest_feasible_arrival_times, optimal_predecessors, optimal_parking, blacklist = main(g)
    print("Finished. Runtime was: {} \n".format(time.time() - start))
    # pick the last solution for f(i,j) and p_i found by the algorithm
    earliest_feasible_arrival_times = {k: v[-1] for k, v in earliest_feasible_arrival_times.items()}
    optimal_parking = {k: v[-1] for k, v in optimal_parking.items()}

initialize the arcs from the source
The iteration over k is currently at:  2
The following nodes move into 2: [1, 3, 5]
[(1, 2), (1, 3), (1, 5), (2, 4), (2, 6), (2, 7), (3, 2), (3, 4), (4, 7), (5, 2), (5, 6), (6, 7), (7, 8)]
[1]
The following nodes move into 3: [1]
f(1, 3) is: 0
1 3 defaultdict(<class 'list'>, {(1, 2): [0], (1, 3): [0], (1, 5): [0]})
Check feasibility for usage of arc ((1, 3)) for earliest feasible time 0
feasible parking times at 1 when moving to 3: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
1st cond satisfied
f(1,3): 0; p_1: 10; t_13: 20
pot_earliest_feasible_arrival_times:  [30]
optimal i for (3,2): 1
[(1, 2), (1, 3), (1, 5), (2, 4), (2, 6), (2, 7), (3, 2), (3, 4), (4, 7), (5, 2), (5, 6), (6, 7), (7, 8)]
[1]
The following nodes move into 5: [1]
f(1, 5) is: 0
1 5 defaultdict(<class 'list'>, {(1, 2): [0], (1, 3): [0], (1, 5): [0], (3, 2): [30]})
Check feasibility for usage of arc ((1, 5)) for earliest feasible time 0
feasible parking times at 1 w

### Solution found by the algorithm with no restriction on parking at node 1

In [11]:
# data wrangling to make solution easier to read
pd.DataFrame.from_dict(optimal_predecessors,  orient='index', columns=["$opt\,  i$"]).merge(
    pd.DataFrame.from_dict(optimal_parking,  orient='index', columns=["$p_i$"]), 
        left_index=True, right_index=True, how='outer').astype('str').merge(
        pd.DataFrame.from_dict(earliest_feasible_arrival_times, orient='index', columns=["$f(j,k)$"]
                              ).replace(float('inf'), 0).astype('int'), 
        left_index=True, right_index=True, how='outer').merge(
        pd.DataFrame.from_dict(blacklist,  orient='index', columns=["blacklisted"]), 
        left_index=True, right_index=True, how='outer').reset_index().rename(
        columns={"index": "$(j,k)$"}).style.set_properties(**{'width': '50px'})

,"$(j,k)$","$opt\, i$",$p_i$,"$f(j,k)$",blacklisted
0,"(1, 2)",nan,nan,0,nan
1,"(1, 3)",nan,nan,0,nan
2,"(1, 5)",nan,nan,0,nan
3,"(2, 4)",1,10,20,None
4,"(2, 6)",1,10,20,None
5,"(2, 7)",3,0,50,"(1, 2)"
6,"(3, 2)",1,15,35,None
7,"(3, 4)",1,15,35,None
8,"(4, 7)",2,9,64,None
9,"(5, 2)",1,15,40,None


The best route is:

In [12]:
get_route_for_arc((7,8), optimal_predecessors)

[(1, 3), (3, 2), (2, 7)]

### Solution found by the algorithm with feasible parking period at the source node 1 restricted to period [0,12].

In [13]:
%%capture 
# capture is used to hide the output of all the print statements. Comment out to see prints

# initialize the graph and change the feasible parking periods for node 1
g = Graph()
g.f_park_t[1] = [0, 12]
# run the algorithm
earliest_feasible_arrival_times, optimal_predecessors, optimal_parking, blacklist = main(g)
# pick the last solution for f(i,j) and p_i found by the algorithm
earliest_feasible_arrival_times = {k: v[-1] for k, v in earliest_feasible_arrival_times.items()}
optimal_parking = {k: v[-1] for k, v in optimal_parking.items()}

In [14]:
# data wrangling to make solution easier to read
pd.DataFrame.from_dict(optimal_predecessors,  orient='index', columns=["$opt\,  i$"]).merge(
    pd.DataFrame.from_dict(optimal_parking,  orient='index', columns=["$p_i$"]), 
        left_index=True, right_index=True, how='outer').astype('str').merge(
        pd.DataFrame.from_dict(earliest_feasible_arrival_times, orient='index', columns=["$f(j,k)$"]
                              ).replace(float('inf'), 0).astype('int'), 
        left_index=True, right_index=True, how='outer').merge(
        pd.DataFrame.from_dict(blacklist,  orient='index', columns=["blacklisted"]), 
        left_index=True, right_index=True, how='outer').reset_index().rename(
        columns={"index": "$(j,k)$"}).style.set_properties(**{'width': '50px'})

,"$(j,k)$","$opt\, i$",$p_i$,"$f(j,k)$",blacklisted
0,"(1, 2)",nan,nan,0,nan
1,"(1, 3)",nan,nan,0,nan
2,"(1, 5)",nan,nan,0,nan
3,"(2, 4)",1,10.0,20,None
4,"(2, 6)",1,10.0,20,None
5,"(2, 7)",1,inf,0,"(1, 2)"
6,"(3, 2)",1,inf,0,"(1, 3)"
7,"(3, 4)",1,inf,0,"(1, 3)"
8,"(4, 7)",2,9.0,64,None
9,"(5, 2)",1,inf,0,None


The best route is:

In [15]:
get_route_for_arc((7,8), optimal_predecessors)

[(1, 2), (2, 6), (6, 7)]